# Checks of input data before context specific GRN inference run
Checks are strongly recommended and can reduce inference pipeline reruns due to errors and therefore save you running time. If you find any error or unexpected output here, change the input files in `data` or `makefiles` folder by regenerating the relevant files in earlier steps, or by editing them by hand. After that, rerun the checks here to make sure everything is expected.

In [1]:
from os.path import join as pjoin
from os.path import exists as pexists
from os import listdir
import logging
import itertools
import numpy as np
import pandas as pd
from dictys.traj import trajectory,point

dirdata='../data'
dirmakefiles='../makefiles'

In [2]:
#Detect whether joint profiles from makefile
with open(pjoin(dirmakefiles,'config.mk'),'r') as f:
	s=f.readlines()
s=[x.strip() for x in s if x.startswith('JOINT=')][-1]
s=s[len('JOINT='):]
if s not in {'0','1'}:
	raise ValueError('Invalid JOINT variable in '+pjoin(dirmakefiles,'config.mk'))
isjoint=s=='1'
print(f'Joint profile: {isjoint}')


Joint profile: True


In [3]:
#Gene & cell names from RNA
namec_rna=pd.read_csv(pjoin(dirdata,'expression.tsv.gz'),header=0,index_col=0,sep='\t',nrows=1)
namec_rna=np.array(namec_rna.columns)
snamec_rna=set(namec_rna)
print(f'Found {len(namec_rna)} cells with RNA profile')
if len(namec_rna)<100:
	raise ValueError('<100 cells found with RNA profile')
nameg=pd.read_csv(pjoin(dirdata,'expression.tsv.gz'),header=0,index_col=0,sep='\t',usecols=[0,1])
nameg=np.array(nameg.index)
snameg=set(nameg)
print(f'Found {len(nameg)} genes with RNA profile')
if len(nameg)<100:
	raise ValueError('<100 genes found with RNA profile')


Found 11898 cells with RNA profile
Found 24036 genes with RNA profile


In [4]:
#Cell names from ATAC
namec_atac=listdir(pjoin(dirdata,'bams'))
namec_atac=np.array(['.'.join(x.split('.')[:-1]) for x in namec_atac if x.endswith('.bam')])
snamec_atac=set(namec_atac)
print(f'Found {len(namec_atac)} cells with ATAC profile')
if isjoint and len(snamec_rna-snamec_atac)>0:
	raise FileNotFoundError('Not all cells with RNA profile has a bam file in bams folder for the joint profiling dataset.')


Found 11898 cells with ATAC profile


In [5]:
#TF names from motifs
with open(pjoin(dirdata,'motifs.motif'),'r') as f:
	nameg_motif=f.readlines()
nameg_motif=[x.split('\t')[1] for x in nameg_motif if x.startswith('>')]
if len(nameg_motif)!=len(set(nameg_motif)):
	raise ValueError('Duplicate motifs found')
print(f'Found {len(nameg_motif)} motifs')
nameg_motif=set([x.split('_')[0] for x in nameg_motif])
print(f'Found {len(nameg_motif)} TFs')
nameg_motif_notfound=sorted(nameg_motif-set(nameg))
nameg_motif=sorted(nameg_motif&set(nameg))
snameg_motif=set(nameg_motif)
print(f'Found {len(nameg_motif)} TFs in current dataset')
print(f'Missing {len(nameg_motif_notfound)} TFs in current dataset: '+','.join(nameg_motif_notfound))
if len(nameg_motif)<10:
	raise ValueError('<10 TFs found')

Found 769 motifs
Found 678 TFs
Found 461 TFs in current dataset
Missing 217 TFs in current dataset: ANDR,AP2A,AP2B,AP2C,AP2D,ARI3A,ARI5B,ATF6A,BARH1,BARH2,BC11A,BHA15,BHE22,BHE23,BHE40,BHE41,BMAL1,BRAC,BSH,COE1,COT1,COT2,CR3L1,CR3L2,ERR1,ERR2,ERR3,EVI1,GCR,HEN1,HMBX1,HME1,HME2,HNF6,HTF4,HXA1,HXA10,HXA11,HXA13,HXA2,HXA5,HXA7,HXA9,HXB1,HXB13,HXB2,HXB3,HXB4,HXB6,HXB7,HXB8,HXC10,HXC11,HXC12,HXC13,HXC6,HXC8,HXC9,HXD10,HXD11,HXD12,HXD13,HXD3,HXD4,HXD8,HXD9,ITF2,KAISO,MCR,MGAP,MLXPL,MYBA,MYBB,NDF1,NDF2,NF2L1,NF2L2,NFAC1,NFAC2,NFAC3,NFAC4,NGN2,NKX21,NKX22,NKX23,NKX25,NKX28,NKX31,NKX32,NKX61,NKX62,ONEC2,ONEC3,OZF,P53,P5F1B,P63,P73,PEBB,PHX2A,PHX2B,PIT1,PKNX1,PLAL1,PO2F1,PO2F2,PO2F3,PO3F1,PO3F2,PO3F3,PO3F4,PO4F1,PO4F2,PO4F3,PO5F1,PO6F1,PO6F2,PRD14,PRGR,RHXF1,RORG,RX,SMCA1,SMCA5,SRBP1,SRBP2,STA5A,STA5B,STF1,SUH,TF2LX,TF65,TF7L1,TF7L2,TFE2,THA,THA11,THB,TWST1,TYY1,TYY2,UBIP1,UNC4,Z324A,Z354A,ZBT14,ZBT17,ZBT18,ZBT48,ZBT49,ZBT7A,ZBT7B,ZEP1,ZEP2,ZF64A,ZKSC1,ZKSC3,ZN121,ZN134,ZN136,ZN140,ZN143,ZN148,Z

In [6]:
#Chromosomes and gene names from bed file
with open(pjoin(dirdata,'gene.bed'),'r') as f:
	nameg_bed=f.readlines()
nameg_bed=[x.strip() for x in nameg_bed]
nameg_bed=[x.split('\t') for x in nameg_bed if len(x)>0]
if len(set([len(x) for x in nameg_bed]))!=1:
	raise ValueError('Unequal number of columns in gene.bed')
nameg_bed=list(filter(lambda x:x[3] in snameg,nameg_bed))
if len(nameg_bed[0])<6:
	logging.warn('No strand information in gene.bed')
namechr_bed=sorted(set([x[0] for x in nameg_bed]))
snamechr_bed=set(namechr_bed)
nameg_bed=[x[3] for x in nameg_bed]
snameg_bed=set(nameg_bed)
if len(nameg_bed)!=len(snameg_bed):
	from collections import Counter
	raise ValueError('Duplicate gene found in gene.bed')
nameg_bed=sorted(nameg_bed)
print(f'Found {len(nameg_bed)} genes with TSS information')
if len(nameg_bed)<100:
	raise ValueError('<100 genes with TSS information')


Found 23533 genes with TSS information


In [7]:
#Ref genome chromosomes
namechr=listdir(pjoin(dirdata,'genome'))
namechr=list(filter(lambda x:x.endswith('.fa'),namechr))
t1=[]
for xi in namechr:
	with open(pjoin(dirdata,'genome',xi),'r') as f:
		t1.append(list(filter(lambda x:x.startswith('>'),f.readlines())))
namechr=[x.strip().strip('>') for x in itertools.chain.from_iterable(t1)]
snamechr=set(namechr)
if len(snamechr_bed-snamechr)>0:
	logging.warning('Chromosomes not found in genome: '+','.join(sorted(snamechr_bed-snamechr)))


In [8]:
#Blacklist chromosomes
if pexists(pjoin(dirdata,'blacklist.bed')):
	with open(pjoin(dirdata,'blacklist.bed'),'r') as f:
		namechr_bl=f.readlines()
	namechr_bl=[x.strip() for x in namechr_bl]
	namechr_bl=[x.split('\t')[0] for x in namechr_bl if len(x)>0]
	snamechr_bl=set(namechr_bl)
	if len(snamechr_bl-namechr_bl)>0:
		logging.warn('Blacklist chromosomes not found in genome: '+','.join(sorted(snamechr_bl-namechr_bl)))


## For context specific GRN inference only

In [9]:
#Subsets
with open(pjoin(dirdata,'subsets.txt'),'r') as f:
	names=f.readlines()
names=[x.strip() for x in names]
names=list(filter(lambda x:len(x)>0,names))

In [10]:
#Cell names in each subset

#Loading cell names
namec_srna=[]
namec_satac=[]
for xi in names:
	with open(pjoin(dirdata,'subsets',xi,'names_rna.txt'),'r') as f:
		t1=f.readlines()
	t1=[x.strip() for x in t1]
	t1=list(filter(lambda x:len(x)>0,t1))
	namec_srna.append(t1)
	with open(pjoin(dirdata,'subsets',xi,'names_atac.txt'),'r') as f:
		t1=f.readlines()
	t1=[x.strip() for x in t1]
	t1=list(filter(lambda x:len(x)>0,t1))
	namec_satac.append(t1)
snamec_srna,snamec_satac=[[set(y) for y in x] for x in [namec_srna,namec_satac]]

#Checking cell names
t1=list(itertools.chain.from_iterable(namec_srna))
if len(t1)!=len(set(t1)):
	print(len(t1),len(set(t1)))
	raise ValueError('Found RNA cells assigned to multiple subsets')
t1=set(t1)
if len(t1-snamec_rna)>0:
	raise ValueError('Subset RNA cells not found in population')
if len(snamec_rna-t1)>0:
	logging.warn('Found RNA cells not assigned to any subset')

if isjoint:
	if any([frozenset(x)!=frozenset(y) for x,y in zip(snamec_srna,snamec_satac)]):
		raise ValueError('Subset assignments must be identical for joint profiles.')
else:
	t1=list(itertools.chain.from_iterable(namec_satac))
	if len(t1)!=len(set(t1)):
		raise ValueError('Found ATAC cells assigned to multiple subsets')
	t1=set(t1)
	if len(t1-snamec_atac)>0:
		raise ValueError('Subset ATAC cells not found in population')
	if len(snamec_atac-t1)>0:
		logging.warn('Found ATAC cells not assigned to any subset')
